# Quality Assessment
Now it's your turn. Follow [the steps on the platform](https://platform.wbscodingschool.com/courses/data-science/9474/) and use what you've learnt to see how reliable the data is.

In [ ]:
import pandas as pd

Load our cleaned DataFrames

In [ ]:
# orders_cl.csv
url = "https://drive.google.com/file/d/1Tla62vfu__kCqvgypZyVt2S9VuC016yH/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders_cl = pd.read_csv(path)

# orderlines_cl.csv
url = "https://drive.google.com/file/d/1OhtkQS2fwOYdzfd-qPh7im35iLc-L9TA/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines_cl = pd.read_csv(path)

# products_cl.csv
url = "https://drive.google.com/file/d/1s7Lai4NSlsYjGEPg1QSOUJobNYVsZBOJ/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products_cl = pd.read_csv(path)

## 1.&nbsp; Define Pandas display format

In [ ]:
# your code here
# set display format
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 1000)

## 2.&nbsp; Exclude unwanted orders

In [ ]:
# your code here
or_df = or_df.drop(or_df.loc[or_df['state']=='Shopping Basket'].index)
or_df = or_df.drop(or_df.loc[or_df['state']=='Cancelled'].index)

## 3.&nbsp; Exclude orders with unknown products


In [ ]:
# your code here
ol_df.loc[ol_df['sku'].isin(list(pr_df['sku'])),'valid_sku'] = True
ol_df.loc[~ol_df['sku'].isin(list(pr_df['sku'])),'valid_sku'] = False
or_df.loc[or_df['id_order'].isin(list(ol_df['id_order'])),'valid_ord'] = True
or_df.loc[~or_df['id_order'].isin(list(ol_df['id_order'])),'valid_ord'] = False
ol_df.loc[ol_df['id_order'].isin(list(or_df['id_order'])),'valid_ord'] = True
ol_df.loc[~ol_df['id_order'].isin(list(or_df['id_order'])),'valid_ord'] = False
ol_df.drop(ol_df.loc[ol_df['valid_sku']==False].index)
or_df = or_df.drop(or_df.loc[or_df['valid_ord']==False].index)
ol_df = ol_df.drop(ol_df.loc[ol_df['valid_ord']==False].index)


## 4.&nbsp; Explore the revenue from different tables

#### Step 1:
Create the `unit_price_total` as `orderlines.unit_price` * `orderlines.product_quantity`

In [ ]:
# your code here
ol_df.loc[:,'unit_price_total'] = ol_df['unit_price'] * ol_df['product_quantity']


#### Step 2: 
Group by `order_id`, summarising by the sum of `unit_price_total`

In [ ]:
# your code here
(
    ol_df.groupby('id_order')
    .agg({'unit_price_total':'sum'})
    .sort_values(by='unit_price_total',ascending=False)
    .head(10)
)

### What is the average difference between `total_paid` and `unit_price_total`?

In [ ]:
# your code here
ol_or_df = ol_df.merge(or_df,on='id_order',how='inner')
ol_or_df.loc[:,'total_paid_per_unit_price'] = ol_or_df['total_paid'] - ol_or_df['unit_price_total']

(
    ol_or_df.groupby(['id_order','unit_price_total','total_paid'])
    .agg({'total_paid_per_unit_price':'mean'})
    .nlargest(10,'total_paid_per_unit_price')
    
)


### What is the distribution of these differences?

In [ ]:
# your code here
(
    ol_or_df.groupby('id_order')
    .agg({'total_paid_per_unit_price':'mean'})
    .nlargest(100,'total_paid_per_unit_price')
    .plot.hist()
)

### Can all the differences be explained by shipping costs? If not, what are other plausible explanations?

In [ ]:
# your explanation here
outliers occur in the range where : diff > 14000 mean difference in order total over line item unit price 
majority of orders have on average lower mean difference
price disparity can be explained by shipping fees of high value items, taxes paid, or applied discount

### If there are differences that you can’t explain: what should you do with these orders?

In [ ]:
# your code here
many orderlines have a unit price of zero.
these should be dropped

## 5.&nbsp; Become confident about your dataset

Let's hear your thoughts about what you discovered.

In [ ]:
# your thoughts & code here
1. promo price not applied on many orderline items
2. orderlines should annotate if promo is applied 
3. maybe separate promo table with date, price, sku
4. apply promo price based on order/orderline condition (date, etc)
5. high orders occur in summer (april - june) and holidays (nov - mar)
6. ranges between order.created_date and orderline.date maybe explained by order state (ie pending)

Do not forget to download your quality controlled DataFrames

In [ ]:
from google.colab import files

#orders_quality_controlled.to_csv("orders_qu.csv", index=False)
#files.download("orders_qu.csv")

#orderlines_quality_controlled.to_csv("orderlines_qu.csv", index=False)
#files.download("orderlines_qu.csv")